<a href="https://colab.research.google.com/github/achnda/git-one/blob/main/breastcancer_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

In [ ]:
data = load_breast_cancer()
train_x, train_y = data.data, data.target

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():

    model = Sequential()

    model.add(Dense(25,input_dim=train_x_scaled.shape[1]))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.8))

    model.add(Dense(12))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))

    model.add(Dense(6))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))

    model.add(Dense(3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=BinaryCrossentropy(), optimizer=RMSprop(), metrics=['accuracy', Precision(), Recall()])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
history = model.fit(train_x_scaled, train_y, epochs=100, batch_size=32)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5802 - loss: 0.8661 - precision_19: 0.6613 - recall_19: 0.7733
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6271 - loss: 0.7877 - precision_19: 0.6676 - recall_19: 0.8399 
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6096 - loss: 0.8418 - precision_19: 0.6676 - recall_19: 0.8191 
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5890 - loss: 0.9200 - precision_19: 0.6248 - recall_19: 0.8365 
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5767 - loss: 0.7995 - precision_19: 0.6248 - recall_19: 0.8015 
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6155 - loss: 0.7804 - precision_19: 0.6427 - recall_19: 0.8715 
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5600 - loss: 0.7170 - precision_19: 0.5920 - recall_19: 0.8511 
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5962 - loss: 0.7623 - precision_1

In [ ]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_x_scaled, test_y)
test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Test F1 Score: {test_f1_score}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9666 - loss: 0.1199 - precision_19: 0.9764 - recall_19: 0.9709  
Test Accuracy: 0.9736841917037964
Test Precision: 0.9857142567634583
Test Recall: 0.9718309640884399
Test F1 Score: 0.9787233789156251
